In [1]:
import datetime
import time
import functools

import pandas as pd
import numpy as np
import pytz

import nba_py
import nba_py.game
import nba_py.player
import nba_py.team

import pymysql
from sqlalchemy import create_engine

from password import hoop_pwd
pwd = hoop_pwd.password

In [2]:
conn = create_engine('mysql+pymysql://root:%s@118.190.202.87:3306/nba_stats' % pwd)

ustz = pytz.timezone('America/New_York')
us_time = datetime.datetime.now(ustz)
print('New York time: ' + str(us_time.date()) + ' ' + str(us_time.time())[:8])

try:
    # read sql table of game header
    game_header = pd.read_sql_table('game_header', conn)
    length_1 = len(game_header)
    print(str(length_1) + ' games loaded.')
    # set begin date to the newest date in sql table
    begin = datetime.datetime.strptime(game_header.iloc[-1]['GAME_DATE_EST'][:10], 
                                       "%Y-%m-%d").date() + datetime.timedelta(days=-2)
except ValueError:
    print('no table yet!')
    length_1 = 0
    # if no table yet, set begin date to 2012-10-29
    begin = datetime.date(2012, 10, 29)
    # grab game headers of begining date
    game_header = nba_py.Scoreboard(month = begin.month, 
                                        day = begin.day, 
                                        year = begin.year, league_id = '00', offset = 0).game_header()

# set end date to us yesterday
end = us_time.date() + datetime.timedelta(days=-1)

for i in range((end - begin).days + 1):
    # grab game headers from begin date to end date
    day = begin + datetime.timedelta(days = i)
    game_header = game_header.append(nba_py.Scoreboard(month = day.month, 
                                                       day = day.day, 
                                                       year = day.year, 
                                                       league_id = '00', 
                                                       offset = 0).game_header())
    print(str(day) + ' finished!    ' + str(datetime.datetime.now().time())[:8])

game_header = game_header[game_header['GAME_STATUS_ID'] == 3]
    
length_2 = len(game_header)
# drop the duplicate by game id
game_header = game_header.drop_duplicates('GAME_ID')
length_3 = len(game_header)
print(str(length_2 - length_3) + ' duplicates droped.')
print(str(length_3 - length_1) + ' games added.')

# sort game headers by game id ascending
# game_header = game_header.sort_values('GAME_ID')

# commit new game headers to sql table
game_header.to_sql('game_header', conn, index = False, if_exists = 'replace')
print(str(length_3) + ' game headers commit complete!')

New York time: 2018-01-12 22:10:11
7181 games loaded.
2017-10-26 finished!    21:10:26
2017-10-27 finished!    21:10:27
2017-10-28 finished!    21:10:29
2017-10-29 finished!    21:10:30
2017-10-30 finished!    21:10:31
2017-10-31 finished!    21:10:32
2017-11-01 finished!    21:10:33
2017-11-02 finished!    21:10:34
2017-11-03 finished!    21:10:36
2017-11-04 finished!    21:10:37
2017-11-05 finished!    21:10:38
2017-11-06 finished!    21:10:39
2017-11-07 finished!    21:10:40
2017-11-08 finished!    21:10:41
2017-11-09 finished!    21:10:42
2017-11-10 finished!    21:10:43
2017-11-11 finished!    21:10:44
2017-11-12 finished!    21:10:46
2017-11-13 finished!    21:10:47
2017-11-14 finished!    21:10:48
2017-11-15 finished!    21:10:49
2017-11-16 finished!    21:10:50
2017-11-17 finished!    21:10:51
2017-11-18 finished!    21:10:52
2017-11-19 finished!    21:10:53
2017-11-20 finished!    21:10:54
2017-11-21 finished!    21:10:55
2017-11-22 finished!    21:10:56
2017-11-23 finished!  

In [3]:
conn = create_engine('mysql+pymysql://root:%s@118.190.202.87:3306/nba_stats' % pwd)

game_stats_logs = pd.DataFrame()

try:
    # read sql table of game stats logs id
    game_stats_logs_id = pd.read_sql_table('game_stats_logs', conn, columns = ['GAME_ID'])
    length_1 = len(game_stats_logs_id)
    print(str(length_1) + ' player stats loaded.')
except ValueError:
    print('no table yet!')
    length_1 = 0
    # create table and commit it to sql
    game_stats_logs.to_sql('game_stats_logs', conn, index = False, if_exists = 'append')
    print('game stats logs initialized!')

# define game types by the head of game id
game_type = {'001': 'pre_season', '002': 'regular_season', '003': 'all_star', '004': 'play_offs'}

# ------method 1------for game id in game headers from the max one in sql table
# for i in game_header[game_header['GAME_ID'] >= game_stats_logs['GAME_ID'].max()]['GAME_ID']:

# ------method 2------for game id in game header but not in game stats logs 
for i in game_header['GAME_ID'][game_header['GAME_ID'].isin(game_stats_logs_id['GAME_ID'].drop_duplicates()) == False]:
    # get game player stats of i
    game_stats = nba_py.game.Boxscore(i).player_stats()
    # create home team player stats
    home_team_id = int(game_header[game_header['GAME_ID'] == i]['HOME_TEAM_ID'])
    home_stats_logs = game_stats[game_stats['TEAM_ID'] == int(home_team_id)].copy()
    home_stats_logs['LOCATION'] = 'HOME'
    home_stats_logs['AGAINST_TEAM_ID'] = int(game_header[game_header['GAME_ID'] == i]['VISITOR_TEAM_ID'])
    # create away team player stats
    away_team_id = int(game_header[game_header['GAME_ID'] == i]['VISITOR_TEAM_ID'])
    away_stats_logs = game_stats[game_stats['TEAM_ID'] == int(away_team_id)].copy()
    away_stats_logs['LOCATION'] = 'AWAY'
    away_stats_logs['AGAINST_TEAM_ID'] = int(game_header[game_header['GAME_ID'] == i]['HOME_TEAM_ID'])
    # combine home and away team player stats and append to game stats logs
    game_stats_logs = game_stats_logs.append(home_stats_logs)
    game_stats_logs = game_stats_logs.append(away_stats_logs)
    print('game ' + i + ' added!    ' + str(datetime.datetime.now().time())[:8])

def min_convert(m):
    '''
    convert mm:ss to float
    '''
    try:
        if ':' in m:
            return float(m[:-3]) + round(float(m[-2:])/60, 2)
        else:
            return float(m)
    except TypeError:
        return None

# create float time
game_stats_logs['MINS'] = game_stats_logs['MIN'].apply(min_convert)
# set 0 time player to None
game_stats_logs['MINS'] = game_stats_logs['MINS'].apply(lambda x: None if x == 0 else x)
# add game type
game_stats_logs['GAME_TYPE'] = game_stats_logs['GAME_ID'].apply(lambda x: x[:3]).map(game_type)
# add game date and game sequence
game_stats_logs = game_stats_logs.merge(game_header[['GAME_DATE_EST', 'GAME_SEQUENCE', 'GAME_ID']], 
                                        how = 'left', on = 'GAME_ID')
# add new ordered game_id
game_stats_logs['GAME_ID_O'] = game_stats_logs['GAME_ID'].apply(lambda x: x[3:5] + x[:3] + x[-5:])

length_2 = len(game_stats_logs)
# drop duplicate game stats by game id and player id
game_stats_logs = game_stats_logs.drop_duplicates(['GAME_ID', 'PLAYER_ID'])
length_3 = len(game_stats_logs)
print(str(length_2 - length_3) + ' duplicates droped.')
print(str(length_3) + ' player stats added.')

# commit new game stats logs to sql table
game_stats_logs.to_sql('game_stats_logs', conn, index = False, if_exists = 'append')
print(str(length_3) + ' player stats commit complete!')

187481 player stats loaded.
game 0011300114 added!    21:12:35
game 0021700085 added!    21:12:36
game 0021700086 added!    21:12:37
game 0021700087 added!    21:12:37
game 0021700088 added!    21:12:38
game 0021700089 added!    21:12:39
game 0021700090 added!    21:12:40
game 0021700091 added!    21:12:41
game 0021700092 added!    21:12:42
game 0021700093 added!    21:12:43
game 0021700094 added!    21:12:44
game 0021700095 added!    21:12:45
game 0021700096 added!    21:12:46
game 0021700097 added!    21:12:46
game 0021700098 added!    21:12:47
game 0021700099 added!    21:12:48
game 0021700100 added!    21:12:50
game 0021700101 added!    21:12:51
game 0021700102 added!    21:12:52
game 0021700103 added!    21:12:53
game 0021700104 added!    21:12:54
game 0021700105 added!    21:12:54
game 0021700106 added!    21:12:55
game 0021700107 added!    21:12:56
game 0021700108 added!    21:12:57
game 0021700109 added!    21:12:58
game 0021700110 added!    21:12:59
game 0021700111 added!    2

game 0021700318 added!    21:16:10
game 0021700320 added!    21:16:10
game 0021700319 added!    21:16:11
game 0021700321 added!    21:16:12
game 0021700322 added!    21:16:13
game 0021700323 added!    21:16:14
game 0021700324 added!    21:16:14
game 0021700325 added!    21:16:15
game 0021700326 added!    21:16:17
game 0021700327 added!    21:16:17
game 0021700328 added!    21:16:18
game 0021700329 added!    21:16:19
game 0021700330 added!    21:16:20
game 0021700331 added!    21:16:21
game 0021700332 added!    21:16:22
game 0021700333 added!    21:16:23
game 0021700334 added!    21:16:24
game 0021700335 added!    21:16:25
game 0021700336 added!    21:16:26
game 0021700337 added!    21:16:27
game 0021700338 added!    21:16:28
game 0021700339 added!    21:16:28
game 0021700340 added!    21:16:29
game 0021700341 added!    21:16:30
game 0021700342 added!    21:16:31
game 0021700343 added!    21:16:32
game 0021700344 added!    21:16:33
game 0021700345 added!    21:16:34
game 0021700346 adde

game 0021700553 added!    21:19:53
game 0021700554 added!    21:19:54
game 0021700555 added!    21:19:55
game 0021700556 added!    21:19:56
game 0021700557 added!    21:19:57
game 0021700558 added!    21:19:58
game 0021700559 added!    21:19:58
game 0021700560 added!    21:19:59
game 0021700561 added!    21:20:00
game 0021700562 added!    21:20:01
game 0021700563 added!    21:20:02
game 0021700564 added!    21:20:03
game 0021700565 added!    21:20:04
game 0021700566 added!    21:20:04
game 0021700567 added!    21:20:05
game 0021700569 added!    21:20:06
game 0021700570 added!    21:20:07
game 0021700568 added!    21:20:08
game 0021700571 added!    21:20:09
game 0021700572 added!    21:20:10
game 0021700573 added!    21:20:11
game 0021700574 added!    21:20:12
game 0021700575 added!    21:20:12
game 0021700576 added!    21:20:13
game 0021700577 added!    21:20:14
game 0021700578 added!    21:20:15
game 0021700579 added!    21:20:16
game 0021700580 added!    21:20:17
game 0021700581 adde